In [2]:
# Importing required packages
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ee
import geemap

In [8]:
pip install pycrs

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycrs: filename=pycrs-1.0.2-py3-none-any.whl size=32735 sha256=d1102f9aacb87c09f5c9497c55115f459b34f222e1db4a53ce3227f09eb55c6b
  Stored in directory: /Users/lauralayton/Library/Caches/pip/wheels/b5/4a/72/1ba05f57ddf2cc80ad21a26512097762561d646ff3ff85f729
Successfully built pycrs

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [65]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1Ab32j93SI4tbt6ZWebmtIuRtSsjj1NgKQ4YHbXhdbslJtDd_0rcFjpkGfm8



Successfully saved authorization token.


In [5]:
os.getcwd()

# Users/lauralayton/Downloads/26914_MaxOpenWater.shp

base_dir = '/Users/lauralayton/Downloads/'

'/Users/lauralayton/Jupyter Notebooks'

In [18]:
in_shp = os.path.join(base_dir, f"26914_MaxOpenWater.shp")
gdf = gpd.read_file(in_shp)
# Reproject RAW input to CRS 32614 
gdf_utm = gdf.to_crs(epsg=32614)

# 10 m OUTWARD
# Read input
# Buffer each feature
gdf_utm["geometry"] = gdf_utm.buffer(10)  # meters
# Reproject back to EPSG:4326 and save
gdf_out = gdf_utm.to_crs("EPSG:4326")
gdf_out.to_file(os.path.join(base_dir, f"WetlandsBuffered_10m.geojson"), driver="GeoJSON")

In [73]:
## --- CREATING THE GRID 

# --- INPUTS ---
# wetlands_shp = os.path.join(base_dir, "26914_MaxOpenWater.shp")
# wetlands_geojson = os.path.join(base_dir, "WetlandsBuffered_10m.geojson")
wetlands_geojson = os.path.join(base_dir, "WetlandsOpenWater_4326_EDITED.geojson")

scale = 30
date = "2015-08-04"

# --- Load wetland polygons ---
fc = geemap.geojson_to_ee(wetlands_geojson)
total = fc.size().getInfo()
geometry_all = fc.geometry()

# --- Prepare list to collect grids ---
grids_list = []

# --- Loop through wetlands ---
for i in range(total):
    wet_feat = ee.Feature(fc.toList(total).get(i))
    wet_id = wet_feat.get("WETLAND_ID").getInfo() if wet_feat.get("WETLAND_ID") else f"Wetland_{i+1}"
    print(f"Processing wetland {i+1}/{total}: {wet_id}")

    aoi = wet_feat.geometry()

    # --- Dynamic World for this AOI ---
    end_date = ee.Date(date).advance(30, "day")
    dw_ic = (ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
             .filterBounds(aoi)
             .filterDate(date, end_date))

    if dw_ic.size().getInfo() == 0:
        print(f"No DW image for {wet_id} — skipping.")
        continue

    dw_single = ee.Image(dw_ic.mosaic().copyProperties(dw_ic.first(), ["system:time_start"]))
    proj_dw = dw_single.select("water").projection().atScale(scale)

    # --- Build pixel-aligned grid for this AOI ---
    pixel_grid = (
        aoi.coveringGrid(proj_dw)
           .map(lambda ft: ft.set("CELL_AREA_M2", ft.geometry().area(1)))
           .map(lambda ft: ft.set("WETLAND_ID", wet_id)))

    # --- Add centroid coordinates ---
    def add_centroid(ft):
        c = ft.geometry().centroid(1)
        xy = c.coordinates()
        return ft.set({
            "LONG": xy.get(0),
            "LAT": xy.get(1)})
    
    pixel_grid = pixel_grid.map(add_centroid)

    # Collect this grid
    grids_list.append(pixel_grid)

# --- Combine all wetlands into one grid ---
if len(grids_list) == 0:
    raise ValueError("No valid wetlands processed — nothing to export.")
    
combined_grid = ee.FeatureCollection(grids_list).flatten()

# --- Export combined grid ---
#out_path = os.path.join(base_dir, "output/Cottonwood/Grid_AllWetlands_Cottonwood.geojson")
out_path = os.path.join(base_dir, "output/Grid_WoW_4326.geojson")
geemap.ee_export_geojson(combined_grid, filename=out_path)

geemap.ee_export_image(dw_single.select("water"), filename=os.path.join(base_dir, "output/_ImageExample_Label.tif"), 
                       scale=30, region=aoi)


Processing wetland 1/8: BL
Processing wetland 2/8: DY
Processing wetland 3/8: FE
Processing wetland 4/8: KE
Processing wetland 5/8: LL
Processing wetland 6/8: MA
Processing wetland 7/8: MO
Processing wetland 8/8: OH
Generating URL ...
Please wait ...
Data downloaded to /Users/lauralayton/Downloads/output/_ImageExample_Label.tif


In [77]:


## --- GETTING GSW DATA

def grid_probs_over_time(grid_geojson_path, start_date, end_date, scale=30):
    grid_fc = geemap.geojson_to_ee(grid_geojson_path)
    aoi = grid_fc.geometry()

    # Get Global Surface Water images in the date range
    gsw_ic = (ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
             .filterBounds(aoi)
             .filterDate(start_date, end_date))

    if gsw_ic.size().getInfo() == 0:
        raise ValueError(f"No Global Surface Water images found between {start_date} and {end_date}.")

    # Function to compute DW probabilities for each date
    def per_image(image):
        image = ee.Image(image)
        date_str = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd")
        proj_dw = image.select("water").projection().atScale(scale)

        # GSW water bands
        gsw_water = image.select('water').toFloat().unmask(0)
        reducer = ee.Reducer.mean().forEachBand(gsw_water)

        # Reduce over the grid cells
        per_cell = gsw_water.reduceRegions(
            collection=grid_fc,
            reducer=reducer,
            scale=scale,
            crs=proj_dw)

        # Add metadata
        return per_cell.map(lambda ft: ft.set({
            "DATE": date_str}))

    # Apply the per-image reducer over the date range
    all_results = ee.FeatureCollection(gsw_ic.map(per_image)).flatten()

    # Export results
    csv_path = os.path.join(base_dir, f"output/_Grid_GSW2_{year}.csv")
    df = geemap.ee_to_df(all_results)
    df.to_csv(csv_path, index=False)

# ---- Run ----
START = '2016-01-1'
END = '2016-12-31'
years = ['2015']

for year in years:
    grid_probs_over_time(grid_geojson_path = os.path.join(base_dir, "output/Grid_WOW_4326.geojson"), 
                     start_date = f'{year}-01-01', 
                     end_date = f'{year}-12-31')
print("done")

done


In [78]:


## --- GETTING AIR TEMP DATA

def grid_probs_over_time(grid_geojson_path, start_date, end_date, scale=30):
    grid_fc = geemap.geojson_to_ee(grid_geojson_path)
    aoi = grid_fc.geometry()

    # Get Global Surface Water images in the date range
    daymet_ic = (ee.ImageCollection("NASA/ORNL/DAYMET_V4")
             .filterBounds(aoi)
             .filterDate(start_date, end_date))

    if daymet_ic.size().getInfo() == 0:
        raise ValueError(f"No Daymet V4 images found between {start_date} and {end_date}.")
    else: 
        print(daymet_ic.size().getInfo())

    # Function to compute DW probabilities for each date
    def per_image(image):
        image = ee.Image(image)
        date_str = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd")
        proj_tmin = image.select("tmin").projection().atScale(scale)
        #proj_tmax = image.select("tmax").projection().atScale(scale)

        # Daymet V4 air temperature bands
        tmin_band = image.select('tmin').toFloat().unmask(0)
        # tmax_band = image.select('tmax').toFloat().unmask(0)
        reducer = ee.Reducer.mean().forEachBand(tmin_band)

        # Reduce over the grid cells
        per_cell = tmin_band.reduceRegions(
            collection=grid_fc,
            reducer=reducer,
            scale=scale,
            crs=proj_tmin)

        # Add metadata
        return per_cell.map(lambda ft: ft.set({
            "DATE": date_str}))

    print("about to apply per-image reducer")
    # Apply the per-image reducer over the date range
    all_results = ee.FeatureCollection(daymet_ic.map(per_image)).flatten()

    # Export results
    csv_path = os.path.join(base_dir, f"output/_Grid2_Daymet_V4_{year}.csv")
    df = geemap.ee_to_df(all_results)
    df.to_csv(csv_path, index=False)
    print("Exported")

# ---- Run ----
START = '2016-01-1'
END = '2016-12-31'
years = [2015]

for year in years:
    grid_probs_over_time(grid_geojson_path = os.path.join(base_dir, "output/Grid_WoW_4326.geojson"), 
                     start_date = f'{year}-01-01', 
                     end_date = f'{str(year + 1)}-01-01')

365
about to apply per-image reducer
Exported


In [79]:


## --- GETTING AIR TEMP DATA

def grid_probs_over_time(grid_geojson_path, start_date, end_date, scale=30):
    grid_fc = geemap.geojson_to_ee(grid_geojson_path)
    aoi = grid_fc.geometry()

    # Get Global Surface Water images in the date range
    daymet_ic = (ee.ImageCollection("NASA/ORNL/DAYMET_V4")
             .filterBounds(aoi)
             .filterDate(start_date, end_date))

    if daymet_ic.size().getInfo() == 0:
        raise ValueError(f"No Daymet V4 images found between {start_date} and {end_date}.")
    else: 
        print(daymet_ic.size().getInfo())

    # Function to compute DW probabilities for each date
    def per_image(image):
        image = ee.Image(image)
        date_str = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd")
        # proj_tmin = image.select("tmin").projection().atScale(scale)
        proj_tmax = image.select("tmax").projection().atScale(scale)

        # Daymet V4 air temperature bands
        # tmin_band = image.select('tmin').toFloat().unmask(0)
        tmax_band = image.select('tmax').toFloat().unmask(0)
        reducer = ee.Reducer.mean().forEachBand(tmax_band)

        # Reduce over the grid cells
        per_cell = tmax_band.reduceRegions(
            collection=grid_fc,
            reducer=reducer,
            scale=scale,
            crs=proj_tmax)

        # Add metadata
        return per_cell.map(lambda ft: ft.set({
            "DATE": date_str}))

    print("about to apply per-image reducer")
    # Apply the per-image reducer over the date range
    all_results = ee.FeatureCollection(daymet_ic.map(per_image)).flatten()

    # Export results
    csv_path = os.path.join(base_dir, f"output/_Grid2_Daymet_V4_tmax_{year}.csv")
    df = geemap.ee_to_df(all_results)
    df.to_csv(csv_path, index=False)
    print("Exported")

# ---- Run ----
START = '2016-01-1'
END = '2016-12-31'
years = [2015]

for year in years:
    grid_probs_over_time(grid_geojson_path = os.path.join(base_dir, "output/Grid_WoW_4326.geojson"), 
                     start_date = f'{year}-01-01', 
                     end_date = f'{str(year + 1)}-01-01')

365
about to apply per-image reducer
Exported
